In [ ]:
##one working after
import deeplake
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load dataset
ds = deeplake.load('hub://activeloop/plantvillage-with-augmentation')
images = ds['images']
labels = ds['labels']

# Function to preprocess and correct labels
def preprocess_and_correct_labels(img, lbl):
    img = tf.image.resize(img, [128, 128])
    img = img / 255.0

    lbl = tf.squeeze(lbl).numpy()

    # Ensure the labels are within the range [0, 37]
    lbl = max(0, min(37, lbl))

    return img, lbl

# TensorFlow data generator
def tf_data_generator():
    for img, lbl in zip(images, labels):
        img_np, lbl_np = preprocess_and_correct_labels(img.numpy(), lbl.numpy())
        yield img_np, lbl_np

# Create the TensorFlow dataset
dataset = tf.data.Dataset.from_generator(
    tf_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

# Augmentation and preprocessing
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, 0.1, 0.2)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

dataset = dataset.map(augment_image)
dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(38, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    dataset,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)]
)

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [20]:
%cd /content/drive/MyDrive/Colab\ Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [21]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [22]:
!git config --global user.name user.name "mohammed_nihall"
!git config --global user.email "mohammednihal9986@gmail.com"

git: 'com!git' is not a git command. See 'git --help'.

The most similar command is
	commit


In [8]:
!git branch -M main

In [9]:
!git remote add origin https://github.com/Mohdnihal03/plant-disease-detection-using-CNN-for-PlantvillageDataset.git

In [11]:
git push -u origin main

SyntaxError: invalid syntax (<ipython-input-11-603ff52d05c2>, line 1)